In [ ]:
!pip install jupyter-dash

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

#webpage

#Google Drive

In [ ]:
# Connectiong to the Google Drive for the permanent storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Checking if the correct version of tensorflow in installed or not 
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
from numpy.random import seed
seed(786)
# Tensor flow for the fast calculation or computation
import tensorflow
tensorflow.random.set_seed(786)

In [ ]:
# Other imports like os for directory management and numpy for mathematical operation on arrays
import os

# Numpy for performing mathematical operations on arrays
import numpy as np

# Importing pandas for data visualization in the forms of tables or data frames
import pandas as pd

# Time for dealing with real time and the pillow for dealing with the image for predictions
from time import strftime

# Pillow for the image used for prediction
from PIL import Image

In [ ]:
# Loading all the files from the directory into variables
X_TRAINING_PATH =  '/content/drive/MyDrive/MNIST/digit_xtrain.csv'
X_TESTING_PATH =   '/content/drive/MyDrive/MNIST/digit_xtest.csv'
Y_TRAINING_PATH =  '/content/drive/MyDrive/MNIST/digit_ytrain.csv'
Y_TESTING_PATH =   '/content/drive/MyDrive/MNIST/digit_ytest.csv'


In [ ]:
%%time
# Geting the wall time required for reading the y-training csv file
y_training_all = np.loadtxt(Y_TRAINING_PATH, delimiter=',', dtype=int)

In [ ]:
y_training_all.shape

In [ ]:
%%time
y_testing = np.loadtxt(Y_TESTING_PATH, delimiter=',', dtype=int)

In [ ]:
%%time

x_training_all = np.loadtxt(X_TRAINING_PATH, delimiter=',', dtype=int)

In [ ]:
%%time

x_testing = np.loadtxt(X_TESTING_PATH, delimiter=',', dtype=int)

# Visualizing the Dataset

In [ ]:
x_training_all.shape

In [ ]:
x_training_all[0]

In [ ]:
y_training_all.shape

In [ ]:
x_testing.shape

In [ ]:
# First 10 labels from training dataset
y_training_all[:10]

In [ ]:
# Our Features are between 0 and 255 which is a large range. So, we have to rescale our training and testing dataset
# After rescaling our data is between 0 and 1
x_training_all = x_training_all / 255.0
x_testing      =  x_testing / 255.0

In [ ]:
x_training_all[0]

In [ ]:
# Our labels are 10 in total from 0 to 9
NR_CLASSES = 10

In [ ]:
y_training_all = np.eye(NR_CLASSES)[y_training_all]

In [ ]:
y_training_all.shape

In [ ]:
y_testing = np.eye(NR_CLASSES)[y_testing]
y_testing.shape

In [ ]:
LOGGING_PATH = '/content/sample_data/MNISTtensorboard_mnist_digit_logs/'

VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
# Total number of features
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS 

In [ ]:
# Now we have to divide our training dataset into smaller training and validation dataset
# Training dataset contains 50000 and avalidation contains 10000

In [ ]:
# From start to the validation size
x_val = x_training_all[:VALIDATION_SIZE]
y_val = y_training_all[:VALIDATION_SIZE]

In [ ]:
x_val.shape

In [ ]:
# From validation size till the end
x_training = x_training_all[VALIDATION_SIZE:]
y_training = y_training_all[VALIDATION_SIZE:]

In [ ]:
x_training.shape

In [ ]:
# Creating tensors
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# tf.placeholder is used for creating tensors 2 parameters (datatype, shape of tensor)
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

In [ ]:
# Hyperparameters
# nr_epochs = 25
# learning_rate = 1e-3
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [ ]:
1e-3

In [ ]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
      """
      Iniatializing the weights
      tf.truncated_normal generates random values execpt for extreme values 
      Shape = no of inputs and neurons in the layer
      Standard Deviation tells how far appart weights should from each others
      """
      initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
      # Calculations for weights 
      w = tf.Variable(initial_value=initial_w, name='W')

      """
      Initializing the biases
      All biases start from same value that is 0
      Shape is no of neurons in layer
      """
      initial_b = tf.constant(value=0.0, shape=bias_dim)
      # Calculation of the biases
      b = tf.Variable(initial_value=initial_b, name='B')

      """
      Input layer of next hidden layer
      MatrixMultiplication
      """
      layer_in = tf.matmul(input, w) + b

      # Checking for the the last hidden layer to apply activation accordingly
      if name=='out':
        # Apply softmax for the last layer
        layer_out = tf.nn.softmax(layer_in)
      else:
        # Apply relu for the remaining layers
        layer_out = tf.nn.relu(layer_in)
        
      tf.summary.histogram('weights', w)
      tf.summary.histogram('biases', b)
        
      return layer_out

In [ ]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [ ]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

#### Defining Loss Function

In [ ]:

with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [ ]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [ ]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [ ]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [ ]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries

In [ ]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

#### Initialise all the variables

In [ ]:

init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [ ]:
size_of_batch = 1000
# size_of_batch = 500
# size_of_batch = 2000


In [ ]:
num_examples = y_training.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

this function is to go to the next batch


In [ ]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Loop

In [ ]:
import array as arr
accuracy_array = []

In [ ]:
%%time
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_training, labels=y_training)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

    accuracy_array.append(batch_accuracy)

print('Done training!')

In [ ]:
# accuracy of the whole training model
accuracy_of_model = np.mean(accuracy_array)
print("Accuracy of model = {}%".format(accuracy_of_model*100))

# Make a Prediction

In [ ]:
im = Image.open('/content/drive/MyDrive/MNIST/2.png')
im

In [ ]:
img = im.resize((28,28))

In [ ]:
bw = img.convert('L')

In [ ]:
img_array = np.invert(bw)

In [ ]:
img_array.shape

In [ ]:
test_img = img_array.ravel()

In [ ]:
test_img.shape

In [ ]:
img_array

In [ ]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [ ]:
print(f'Prediction for test image is {prediction}')

# **Testing and Evaluation**

In [ ]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_testing, Y:y_testing})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

In [ ]:
def predict_test_image(Img_url):
  im = Image.open(Img_url)
  img = im.resize((28, 28))
  bw = img.convert('L')
  img_array = np.invert(bw)
  test_img = img_array.ravel()
  prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
  print(f'Prediction for test image is {prediction}')

In [ ]:
def predict_test_image1(Img_url):
  im = Image.open(Img_url)
  img = im.resize((28, 28))
  bw = img.convert('L')
  img_array = np.invert(bw)
  test_img = img_array.ravel()
  prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
  print(f'Prediction for test image is {prediction}')

##Prediction 1

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/1.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/1.png');


##Prediction 2

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/2.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/2.png');

##Prediction 3

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/3.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/3.png');

##Prediction 4

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/4.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/4.png');

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/ii_4.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/ii_4.png');

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/8.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/8.png');

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/7777.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/7777.png');

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/22222.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/22222.png');

In [ ]:
img = Image.open('/content/drive/MyDrive/MNIST/555555.png')
display(img)
predict_test_image('/content/drive/MyDrive/MNIST/555555.png');

#Webpage

In [ ]:
temp = test_accuracy*100
model_accuracy = round(temp,2)
df = px.data.tips()# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("ML SEMESTER PROJECT"),
    html.H1("DIGIT RECOGNITION"),
    html.H2("Model Accuracy: {}%".format(model_accuracy)),
    html.H2("Img:"),
    #html.Img(src = "https://drive.google.com/uc?export=view&id=1UcMUe3ifP9ijRuQtHWavWsvVnPteRm6w"), #Test image 1
    html.Img(src = "https://drive.google.com/uc?export=view&id=1_L1CCgQ-x1c5FCeExEPmXoXmBpnCUy_W"), #Test image 2
    
    html.H2("Prediction: {}".format(prediction)),
])

# app.run_server(mode='external')
app.run_server(host="127.0.0.1", port="8000")

# Reset for the Next Run

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()